## Synthesize and publish your data to your ArcGIS Online Account
Throughout our analysis, we've created a number of outputs. We won't have sufficient time to explore visualization techniques available in Python to view our different outputs. However, this notebook reveals how we can easily upload and document our datasets to our ArcGIS Online account, where we can more easily create web maps, story maps, and/or dashboards that we can share. 

The workflow, which relies heavily on ESRI's ArcGIS Python API, is fairly simple:
1. Create a `gis` object, authenticated to our NSOE ArcGIS Online Account
* Add our shapefile to the coding environment as a spatially enabled dataframe (aka "GeoAccessor" object)
* Use the [to_featurelayer()](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.GeoAccessor.to_featurelayer) function to publish our feature layer to our NSOE AGOL space
* Optionally: We can update information about our newly published feature layer
* Optionally: We can share out newly published feature layer

First, however, we may want to synthesize our different outputs. These may include all most of the following:
* Exits, tagged with flood hazard zone data
* Exits, tagged with distance from electric substation [*Note: I've not included that here, as I hope you can do this on your own*]
* Exits, tagged with Euclidean distance to nearest DC fast charger location
* Exits, tagged with # of amenities within 1 mile
* Exits, tagged with Drive time distance to nearest DC fast charger location


#### Import packages and create an authenticated link to your AGOL account

In [ ]:
#Ignore warnings
import warnings
warnings.simplefilter('ignore')

In [ ]:
#Load packages
from arcgis import GIS
import pandas as pd
from arcgis.features import GeoAccessor

In [ ]:
#Authenticate our GIS object
gis = GIS(url='https://NSOE.maps.arcgis.com',username='jpfay_790')

#### Import, tidy, and join datasets
The various exit datasets we created should all still have a common `FID` attribute. We can use this common attribute to join each separate exit attribute to a single "master" feature class, which we will then save locally and/or publish to our AGOL account.

In [ ]:
#Read local datasets into spatial dataframes
sdf_Exits = GeoAccessor.from_featureclass('../Data/MJBA/Exits.shp')
sdf_floods = GeoAccessor.from_featureclass('../Data/processed/exits_flood.shp')
sdf_EucDist = GeoAccessor.from_featureclass('../Data/processed/Exits_distance_to_DCFC.shp')
sdf_AmenityCount = GeoAccessor.from_featureclass('../Data/processed/exits_amenity_counts.shp')
sdf_DriveDist = GeoAccessor.from_featureclass('../Data/processed/exits_distance.shp')

In [ ]:
#Subset fields from the Exits feature class and set FID to the index
sdf_Exits = sdf_Exits[['FID','Longitude', 'Latitude', 'County', 'State','SHAPE']].set_index('FID')

In [ ]:
#Extract just the FID and meaningful attribute, setting the index to FID
sdf_floods = sdf_floods[['FID','Flood_Zone']].set_index('FID')
sdf_EucDist = sdf_EucDist[['FID','Dist_to_DC']].set_index('FID')
sdf_AmenityCount = sdf_AmenityCount[['FID','amenities']].set_index('FID')
sdf_DriveDist = sdf_DriveDist[['FID','distance']].set_index('FID')

In [ ]:
#Merge all datasets into a single exit dataset
sdf_Master = pd.merge(left=sdf_Exits,right=sdf_floods,how='left',left_index=True,right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_EucDist,how='left',left_index=True,right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_AmenityCount,how='left',left_index=True,right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_DriveDist,how='left',left_index=True,right_index=True)

In [ ]:
#Rename columns
sdf_Master.rename(columns={'Dist_to_DC':'Mi_to_DCFC',
                           'amenities':'No_amenities',
                           'distance':'RdMi_to_DCFC'},inplace=True)

In [ ]:
#Optional: Save to a new local shapefile
sdf_Master.spatial.to_featureclass('../Data/processed/Exits_final.shp')

#### Publish to your ArcGIS Online account
We've created three new datasets that don't already live on ArcGIS online: the NREL DCFC charging sites, the extracted OSM amenities locations, and of course our attributed candidate sites (i.e. the exits). Let's publish each of these to our AGOL account as hosted feature services. This is easily done using the `to_featurelayer()` function. 

In [ ]:
#Fetch the NREL CSV into a pandas dataframe, then convert to a spatial dataframe
DCFC_df = pd.read_csv('../Data/NREL/DCFC.csv')
DCFC_sdf = GeoAccessor.from_xy(df = DCFC_df, x_column='longitude', y_column='latitude',sr=4326)

In [ ]:
#Convert the sdf to a FeatureLayer, which publishes it to AGOL
DCFC_flyr = DCFC_sdf.spatial.to_featurelayer(title='NREL DC Fast Charger sites',
                                             tags='EV Charger,ENV790')
DCFC_flyr

Click on the link above to view your service on ArcGIS On-line. You'll likely have to log-on as this feature service has not been shared with anyone, yet.

Next, we'll update some properties for this layer. You can certainly do this via the web site, but we'll do this programmatically. We do this by first creating a dictionary of properties and their values and then updating our feature layer with these properties. See: https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.update for more information on what we can update and how the dictionary is structured.

In [ ]:
#Construct the dictionary of feature layer properties
featLayer_properties = {
    'snippet':"NREL DC Fast Charging site. Extracted and subset via the NREL API",
    'summary':"These data were extracted from the NREL API, subset using the following criteria..."
}
#Apply the changes
DCFC_flyr.update(featLayer_properties)

In [ ]:
#Have a second look at our local view
DCFC_flyr

Finally, we want to share this layer with everone in our organization. 

In [ ]:
#Share with everyone in the organization
DCFC_flyr.share(org=True)

In [ ]:
sdf_floods['Flood_Zone'].unique()

# Your assignment:
Publish, document, and share your remaining two datasets: the OSM amenities and the candidate exit sites. You may also want to publish your road network dataset.

With your data now hosted on ArcGIS online, you can create and share web maps of your data. Create a web map. Add your data to this web map. Also add the HILFD substations feature layer you found earlier on AGOL. Create multiple instances of your exits data, each symbolized to show the different criteria. Lastly, show the exits that meet a specific critera by filtering them for records that meet these criteria:
* Is found within an area of 'Area of Minimal Flood Hazard'
* At least 1 amenty found within 1 mile
* At least 50 drive-miles from the nearest DCFC
* No more than 3 miles from a power substation
Display the resulting candidate locations symbolized by # of amenities.

When complete, save and share your WebMap with everyone. Record the URL for the web map and submit to Sakai.